In [1]:
from pyspark import SparkContext
from pyspark import SparkConf

In [2]:
spark

In [3]:
dataPath = "/FileStore/tables/Restaurant_Scores___LIVES_Standard-29ce7.csv"
restaurant = spark.read.format("com.databricks.spark.csv")\
  .option("header","true")\
  .option("inferSchema", "true")\
  .load(dataPath)

In [4]:
display(restaurant)

In [5]:
restaurant.select("business_name").distinct().show(100, False)

In [6]:
#check how many restaurants got particular score

display(restaurant.select("inspection_score").groupBy("inspection_score").count().orderBy("inspection_score", ascending = False))

In [7]:
#to check where the restaurants are located

display(restaurant.select("business_postal_code").groupBy("business_postal_code").count().orderBy("count", ascending = False))

In [8]:
restaurant.printSchema()

In [9]:
import pandas as pd

#reest = pd.read_csv("dbfs:/FileStore/tables/Restaurant_Scores___LIVES_Standard-29ce7.csv")
resta = sqlContext.read.format("csv").load("/FileStore/tables/Restaurant_Scores___LIVES_Standard-29ce7.csv")

#resta = pd.read_csv("/FileStore/tables/Restaurant_Scores___LIVES_Standard-29ce7.csv",sep=',')

In [10]:
from pyspark.sql.functions import *

from_pattern1 = "MM/dd/yyyy HH:mm:ss"
to_pattern1 = "yyyy/MM/dd HH:mm:ss aa"

rest = restaurant \
  .withColumn('inspection_time', unix_timestamp(restaurant['inspection_date'], from_pattern1).cast("timestamp")) \
  .drop('inspection_date')

#rest = restaurant.withColumn("business_inspection_time", unix_timestamp(restaurant["business_inspection_date"],from_pattern1).cast("timestamp")).drop("business_inspection_date")

In [11]:
rest.printSchema()

In [12]:
display(rest)

In [13]:
rest.select(year('inspection_time')).distinct().orderBy('year(inspection_time)').show()

In [14]:
rest.select(month('inspection_time')).distinct().orderBy('month(inspection_time)').show()

In [15]:
#to see on what days inspection took place

rest.filter(year('inspection_time') == '2018').filter(dayofyear('inspection_time') >= 1).filter(dayofyear('inspection_time') <= 32).select(dayofyear('inspection_time')).distinct().orderBy('dayofyear(inspection_time)').show()

In [16]:
rest.filter(year('inspection_time') == '2018').filter(dayofyear('inspection_time') >= 
32).groupBy(dayofyear('inspection_time')).count().orderBy('dayofyear(inspection_time)').show()

In [17]:
display(rest.filter(year('inspection_time') == '2018').filter(month('inspection_time') >= 1).groupBy(month('inspection_time')).count().orderBy('month(inspection_time)'))

In [18]:
display(rest.filter(year('inspection_time') == '2017').filter(month('inspection_time') >= 1).groupBy(month('inspection_time')).count().orderBy('month(inspection_time)'))

In [19]:
display(rest.filter(year('inspection_time') == '2016').filter(month('inspection_time') >= 1).groupBy(month('inspection_time')).count().orderBy('month(inspection_time)'))

In [20]:
display(rest.filter(year('inspection_time') == '2018').filter(dayofyear('inspection_time') >= 1).filter(dayofyear('inspection_time') <= 31).groupBy(dayofyear('inspection_time')).count().orderBy('dayofyear(inspection_time)'))

In [21]:
display(restaurant.select("inspection_score").groupBy("inspection_score").count().orderBy("inspection_score", ascending = False))

In [22]:
%sql

select business_postal_code, sum(inspection_score) 
from restaurant_scores
group by business_postal_code 
order by sum(inspection_score) desc;

In [23]:
%sql

select business_name, business_postal_code from restaurant_scores 
where business_name like "Pizza%"
order by business_name;

In [24]:
%sql
select business_postal_code, inspection_score from restaurant_scores where inspection_score >= 95;